In [ ]:
%pip install yfinance
%pip install pandas
%pip install numpy

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime


ticker = "AAPL"
start_date = "2023-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")


try:
    data = yf.download(
        tickers=ticker,
        start=start_date,
        end=end_date,
        progress=True  # 显示下载进度条
    )

    # 确保数据不为空
    if data.empty:
        raise ValueError("没有获取到数据，请检查股票代码和日期范围")

except Exception as e:
    print(f"下载数据时出错: {e}")
    exit()


data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data['SMA_50'] = data['Close'].rolling(window=50, min_periods=1).mean()

print(data)


[*********************100%***********************]  1 of 1 completed

Price             Open        High         Low       Close     Volume  \
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL   
Date                                                                    
2023-01-03  128.613985  129.226052  122.582119  123.470612  112117500   
2023-01-04  125.267347  127.014716  123.480495  124.744125   89113600   
2023-01-05  125.504260  126.136075  123.164572  123.421242   80962700   
2023-01-06  124.398597  128.623856  123.292916  127.962425   87754700   
2023-01-09  128.801572  131.703978  128.228987  128.485657   70790800   
...                ...         ...         ...         ...        ...   
2025-05-23  193.669998  197.699997  193.460007  195.270004   78432900   
2025-05-27  198.300003  200.740005  197.429993  200.210007   56288500   
2025-05-28  200.589996  202.729996  199.899994  200.419998   45339700   
2025-05-29  203.580002  203.809998  198.509995  199.949997   51477900   
2025-05-30  199.369995  201.960007  196.779999  200

## 成本计算

In [ ]:
def calculate_trade_cost(order):
    """计算交易总成本"""
    # 佣金
    commission = max(order.shares * 0.005, 1)  # $0.005/股 最低$1

    # 滑点模型（基于流动性）
    liquidity_factor = 0.0005 * (order.shares / 10000)**0.5
    slippage = order.price * liquidity_factor

    # 市场冲击（大额订单）
    impact = 0.0002 * order.shares if order.shares > 10000 else 0

    total_cost = commission + slippage + impact
    return total_cost

In [ ]:
def calculate_vwap(df, period='1D'):
    """
    专业级VWAP计算函数
    :param df: 包含OHLCV的DataFrame
    :param period: 计算周期 ('1D'=日, '1W'=周等)
    :return: 添加VWAP列的DataFrame
    """
    # 创建副本避免修改原始数据
    data = df.copy()

    # 转换时间周期
    if period != '1D':
        data = data.resample(period).agg({
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        })

    # 计算典型价格
    data['TypicalPrice'] = (data['High'] + data['Low'] + data['Close']) / 3

    # 计算成交金额
    data['DollarVolume'] = data['TypicalPrice'] * data['Volume']

    # 计算累积值
    data['CumulativeDV'] = data['DollarVolume'].cumsum()
    data['CumulativeVol'] = data['Volume'].cumsum()

    # 计算VWAP
    data['VWAP'] = data['CumulativeDV'] / data['CumulativeVol']

    # 添加VWAP标准差通道
    rolling_std = data['VWAP'].rolling(window=20).std()
    data['VWAP_Upper'] = data['VWAP'] + rolling_std * 2
    data['VWAP_Lower'] = data['VWAP'] - rolling_std * 2

    return data

In [ ]:
def sharpe_ratio(returns, risk_free_rate=0.03, periods=252):
    """
    计算年化夏普比率
    :param returns: 日收益率序列
    :param risk_free_rate: 年化无风险利率
    :param periods: 年化交易日数(股票=252, 加密货币=365)
    :return: 年化夏普比率
    """
    excess_returns = returns - risk_free_rate/periods
    return np.sqrt(periods) * np.mean(excess_returns) / np.std(excess_returns)

# 使用示例
daily_returns = np.array([0.01, -0.005, 0.015, ...])  # 日收益率
sharpe = sharpe_ratio(daily_returns)

In [ ]:
def max_drawdown(values):
    """
    计算最大回撤
    :param values: 资产净值序列
    :return: (最大回撤幅度, 回撤开始点, 回撤结束点)
    """
    peak = values[0]
    max_dd = 0
    start_idx = end_idx = 0
    peak_idx = 0

    for i in range(1, len(values)):
        if values[i] > peak:
            peak = values[i]
            peak_idx = i
        else:
            dd = (peak - values[i]) / peak
            if dd > max_dd:
                max_dd = dd
                start_idx = peak_idx
                end_idx = i

    return max_dd, start_idx, end_idx

In [ ]:
def sortino_ratio(returns, target_return=0, periods=252):
    """
    计算年化索提诺比率
    :param returns: 日收益率序列
    :param target_return: 年化目标收益率
    :param periods: 年交易日数
    :return: 年化索提诺比率
    """
    # 转换年化目标到日目标
    daily_target = (1 + target_return)**(1/periods) - 1

    # 计算下行偏差
    downside_returns = returns[returns < daily_target] - daily_target
    downside_deviation = np.sqrt(np.mean(downside_returns**2))

    # 年化处理
    excess_return = np.mean(returns) * periods - target_return
    return excess_return / (downside_deviation * np.sqrt(periods))

In [ ]:
def comprehensive_risk_report(returns, values, risk_free_rate=0.03):
    """生成综合风险评估报告"""
    sharpe = sharpe_ratio(returns, risk_free_rate)
    sortino = sortino_ratio(returns, risk_free_rate)
    max_dd, start, end = max_drawdown(values)
    calmar = np.mean(returns)*252 / max_dd

    # 风险评估矩阵
    risk_matrix = {
        "夏普比率": {"value": sharpe, "threshold": 1.0},
        "索提诺比率": {"value": sortino, "threshold": 1.5},
        "最大回撤": {"value": max_dd, "threshold": 0.15},
        "卡玛比率": {"value": calmar, "threshold": 0.7}
    }

    # 生成评估报告
    report = "===== 综合风险评估报告 =====\n"
    for metric, data in risk_matrix.items():
        status = "通过" if data['value'] >= data['threshold'] else "警告"
        report += f"{metric}: {data['value']:.4f} ({status})\n"

    # 添加最大回撤详情
    report += f"\n最大回撤详情: {max_dd*100:.2f}% " \
              f"从第{start}天到第{end}天"

    # 整体评估
    passed = sum(1 for data in risk_matrix.values()
                 if data['value'] >= data['threshold'])
    overall = "优秀" if passed >= 3 else "合格" if passed >=2 else "需改进"
    report += f"\n\n综合评级: {overall}"

    return report

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

class Backtester:
    """基本回测框架"""

    def __init__(self, initial_capital=100000):
        """
        初始化回测器
        :param initial_capital: 初始资金
        """
        self.initial_capital = initial_capital
        self.data = None
        self.signals = None
        self.positions = None
        self.portfolio = None
        self.trade_log = []
        self.performance = {}

    def load_data(self, data_path):
        """
        加载历史数据
        :param data_path: 数据文件路径
        """
        # 加载数据
        self.data = pd.read_csv(data_path, parse_dates=['Date'], index_col='Date')

        # 数据验证
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        if not all(col in self.data.columns for col in required_cols):
            raise ValueError("数据必须包含OHLCV列")

        print(f"数据加载成功: {len(self.data)}条记录, {self.data.index.min()} 至 {self.data.index.max()}")

    def generate_signals(self, strategy_func, **params):
        """
        生成交易信号
        :param strategy_func: 策略函数
        :param params: 策略参数
        """
        if self.data is None:
            raise ValueError("请先加载数据")

        # 复制数据避免修改原始数据
        signals_df = self.data.copy()

        # 应用策略函数
        signals_df = strategy_func(signals_df, **params)

        # 验证信号格式
        if 'signal' not in signals_df.columns:
            raise ValueError("策略函数必须生成'signal'列")

        self.signals = signals_df
        print("信号生成完成")

    def execute_backtest(self, commission=0.001, slippage=0.0005):
        """
        执行回测
        :param commission: 佣金率(百分比)
        :param slippage: 滑点率(百分比)
        """
        if self.signals is None:
            raise ValueError("请先生成信号")

        # 初始化持仓和投资组合
        positions = pd.DataFrame(index=self.signals.index).fillna(0.0)
        portfolio = pd.DataFrame(index=self.signals.index)

        # 创建仓位列 (1=做多, 0=空仓, -1=做空)
        positions['position'] = self.signals['signal']

        # 初始化投资组合价值
        portfolio['holdings'] = positions['position'].multiply(
            self.signals['Close'], axis=0)

        # 计算持仓变化
        positions['trade'] = positions['position'].diff()

        # 计算现金变化
        portfolio['cash'] = self.initial_capital - (
            positions['trade'].multiply(self.signals['Close'], axis=0).cumsum()

        # 应用交易成本
        trade_costs = positions['trade'].abs().multiply(
            self.signals['Close'], axis=0) * (commission + slippage)
        portfolio['cash'] -= trade_costs.cumsum()

        # 计算总投资组合价值
        portfolio['total'] = portfolio['cash'] + portfolio['holdings']

        # 计算收益率
        portfolio['returns'] = portfolio['total'].pct_change()

        self.positions = positions
        self.portfolio = portfolio
        print("回测执行完成")

        # 记录交易日志
        self._create_trade_log()

        # 计算绩效指标
        self._calculate_performance()

    def _create_trade_log(self):
        """创建详细的交易日志"""
        trade_dates = self.positions[self.positions['trade'] != 0].index

        for date in trade_dates:
            trade_type = "买入" if self.positions.loc[date, 'trade'] > 0 else "卖出"
            price = self.signals.loc[date, 'Close']
            position = self.positions.loc[date, 'position']
            cash = self.portfolio.loc[date, 'cash']
            total = self.portfolio.loc[date, 'total']

            self.trade_log.append({
                'date': date,
                'type': trade_type,
                'price': price,
                'position': position,
                'cash': cash,
                'total_value': total
            })

    def _calculate_performance(self):
        """计算关键绩效指标"""
        if self.portfolio is None:
            raise ValueError("请先执行回测")

        # 总收益率
        total_return = (self.portfolio['total'][-1] / self.initial_capital) - 1

        # 年化收益率
        days = (self.portfolio.index[-1] - self.portfolio.index[0]).days
        annualized_return = (1 + total_return) ** (365/days) - 1

        # 波动率 (年化)
        volatility = self.portfolio['returns'].std() * np.sqrt(252)

        # 夏普比率 (假设无风险利率3%)
        sharpe_ratio = (annualized_return - 0.03) / volatility

        # 最大回撤
        wealth_index = self.portfolio['total']
        previous_peaks = wealth_index.cummax()
        drawdowns = (wealth_index - previous_peaks) / previous_peaks
        max_drawdown = drawdowns.min()

        # 交易次数
        trade_count = len([t for t in self.trade_log if t['type'] in ['买入', '卖出']])

        self.performance = {
            'total_return': total_return,
            'annualized_return': annualized_return,
            'volatility': volatility,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'trade_count': trade_count
        }

    def plot_results(self):
        """可视化回测结果"""
        if self.portfolio is None:
            raise ValueError("请先执行回测")

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

        # 绘制资产曲线
        ax1.plot(self.portfolio['total'], label='投资组合价值', color='b')
        ax1.set_title('投资组合表现')
        ax1.set_ylabel('价值 (元)')
        ax1.grid(True)
        ax1.legend()

        # 绘制每日收益率
        ax2.plot(self.portfolio['returns'], label='日收益率', color='g', alpha=0.7)
        ax2.set_title('每日收益率')
        ax2.set_ylabel('收益率')
        ax2.set_xlabel('日期')
        ax2.grid(True)

        plt.tight_layout()
        plt.show()

    def print_performance(self):
        """打印绩效报告"""
        if not self.performance:
            raise ValueError("绩效数据未计算")

        print("\n" + "="*50)
        print("策略绩效报告")
        print("="*50)
        print(f"初始资金: \t{self.initial_capital:,.2f}元")
        print(f"最终价值: \t{self.portfolio['total'][-1]:,.2f}元")
        print(f"总收益率: \t{self.performance['total_return']*100:.2f}%")
        print(f"年化收益率: \t{self.performance['annualized_return']*100:.2f}%")
        print(f"年化波动率: \t{self.performance['volatility']*100:.2f}%")
        print(f"夏普比率: \t{self.performance['sharpe_ratio']:.2f}")
        print(f"最大回撤: \t{self.performance['max_drawdown']*100:.2f}%")
        print(f"交易次数: \t{self.performance['trade_count']}次")
        print("="*50)

    def print_trade_log(self, num_trades=10):
        """打印交易日志"""
        print("\n交易日志 (最近{}笔):".format(num_trades))
        print("-"*70)
        print("{:<12} {:<8} {:<10} {:<10} {:<15} {:<15}".format(
            "日期", "操作", "价格", "仓位", "现金", "总资产